# Load HMI data into lvl1 and lvl2 datacubes

In [31]:
import matplotlib.pyplot as plt
import numpy as np

import os
import natsort  # Sorting package. Used when I parse the folder for files. There may be simpler ways to do this.

import astropy.units as u
import astropy.wcs
from astropy.coordinates import SkyCoord, SpectralCoord
import datetime

import stokespy

import sunpy
from sunpy.net import Fido, attrs

# Set working directory. 
work_dir = os.getcwd()

#data_dir = work_dir + '/Data/SDO/'
data_dir = '/home/gabriel/Desktop/Science/StokesPY' + '/Data/SDO/'

script_name = 'HMI_data_loading'

from ndcube import NDCube, GlobalCoords, ExtraCoords

In [2]:
def parse_folder(dir_path=None, inst=None, wave=None, ext=None, 
                 series=None, repo='JSOC', show=True):
    """
    Search all the filenames in a folder containing SDO data and use the keywords
    to select a desired subset. For example setting inst="hmi" will obtain all the hmi filenames.
    dir_path: Path to the directory containing the data.
    inst: SDO instrument
    wave: wavelength (primarily for AIA data)
    ext: Select the file extension
    series: String characterizing the data series (e.g. hmi.S_720s, aia.lev1_euv_12s)
    repo: Choose the data repository. Each repository stores filenames with different syntaxes.
    show: Flag that enumerates the files found.
    """
    
    if dir_path is None:
        dir_path = os.getcwd()
    
    # Read and sort all the filenames in the folder.
    all_fnames = natsort.natsorted(os.listdir(dir_path)) 
    
    # Select a subset of files
    use_fnames = []
    
    for i, file_ in enumerate(all_fnames):
        if repo == 'JSOC':
            sfile_ = file_.lower().split(".")
        elif repo == 'VSO':
            sfile_ = file_.lower().split("_")
        
        if sfile_[0] == inst.lower() and sfile_[1] == series.lower(): 
            use_fnames.append(dir_path + file_)
        
        '''
        if ext is None and inst is None:
            use_fnames.append(dir_path + file_)
        elif ext is None and inst is not None and sfile_[0] == inst.lower() and wave is None:
            use_fnames.append(dir_path + file_)
        elif ext is None and inst is not None and sfile_[0] == inst.lower() and wave is not None and sfile_[2] == wave + 'a':
            use_fnames.append(dir_path + file_)
        elif ext is not None and file_.endswith(ext) and inst is None:
            use_fnames.append(dir_path + file_)
        elif ext is not None and file_.endswith(ext) and inst is not None and sfile_[0] == inst.lower() and wave is None:
            use_fnames.append(dir_path + file_)
        elif ext is not None and file_.endswith(ext) and inst is not None and sfile_[0] == inst.lower() and wave is not None and sfile_[2] == wave + 'a':
            use_fnames.append(dir_path + file_)
        '''
    '''
    for i, file_ in enumerate(all_fnames):
        # Select by extension.
        if ext is not None and file_.endswith(ext):        
            sfile_ = file_.split("_")
            # Select by instrument.
            if inst is not None and sfile_[0] == inst.lower(): 
                if wave is not None and inst.lower() == 'aia' and sfile_[2] == wave + 'a':   
                    use_fnames.append(dir_path + file_)
        else:
            sfile_ = file_.split("_")
            # Select by instrument.
            if inst is not None and sfile_[0] == inst.lower(): 
                if wave is not None and inst.lower() == 'aia' and sfile_[2] == wave + 'a':   
                    use_fnames.append(dir_path + file_)
    '''
    
    if show:
        for i, file_ in enumerate(use_fnames):
            print(i, file_)
    
    return use_fnames

# Test the functiom.
'''
# Parse the data folder and select subset of files.
inst = 'aia'
wave = '171'

use_fnames = parse_folder(dir_path = img_dir, ext='fits', inst='aia', wave='171')
print(len(use_fnames))
for i in use_fnames:
    print(i)
'''

"\n# Parse the data folder and select subset of files.\ninst = 'aia'\nwave = '171'\n\nuse_fnames = parse_folder(dir_path = img_dir, ext='fits', inst='aia', wave='171')\nprint(len(use_fnames))\nfor i in use_fnames:\n    print(i)\n"

In [261]:
def get_HMI_data(user_date, user_notify='gdima@hawaii.edu', user_dir=None, max_conn=1, download=True, show_files=False):
    """
    Function that finds the nearest HMI 720s Stokes data series and inversion results.
    Parameters
    ----------
    user_data: `astropy.time` object.
    user_notify: Notification email. This must be registered with JSOC. 
    """
    
    # Calculate a 1s bounding time around the input date user_date
    # FIDO finds all series where at least one observation was present in the 
    # time interval.
    time0 = astropy.time.Time(user_date.gps - 1., format='gps', scale='tai')
    time1 = astropy.time.Time(user_date.gps + 1., format='gps', scale='tai')

    a_time = attrs.Time(time0, time1)
    print('Time window used for the search: ', a_time)
    
    # Set the notification email. This must be registered with JSOC. 
    a_notify = attrs.jsoc.Notify(user_notify)
    
    # Set the default data directory.
    if user_dir is None:
        user_dir = work_dir + '/Data/SDO/'
    
    # Check if the data directory exists and create one if it doesn't.
    if not os.path.exists(user_dir):
        print('Data directory created: ', user_dir)
        os.makedirs(user_dir)
    
    ### Get the 720s HMI Stokes image series ###
    a_series = attrs.jsoc.Series('hmi.S_720s')
    
    if download:
        results_stokes = Fido.search(a_time, a_series, a_notify)
        down_files = Fido.fetch(results_stokes, path=user_dir, max_conn=1)
        # Sort the input names
        all_fnames_stokes = natsort.natsorted(down_files)
    else:
        all_fnames_stokes = parse_folder(dir_path=data_dir, inst='hmi', series='S_720s', ext='fits', show=show_files)
    
        if len(all_fnames_stokes) > 1:
            tstamps = [i.split('.')[2] for i in all_fnames_stokes]
            tstamps = [sunpy.time.parse_time('_'.join(i.split('_')[0:2])) for i in tstamps]
            tstamps_diff = [np.abs(i.gps - user_date.gps) for i in tstamps]
       
            
        # Search for the closest timestamp
        tstamps_diff = np.asarray(tstamps_diff)
        tstamps_ix, = np.where(tstamps_diff == tstamps_diff.min())

        all_fnames_stokes = np.asarray(all_fnames_stokes)[tstamps_ix]
        
        print(f'No download requested. Nearest {len(all_fnames_stokes)} Stokes files found: ')
        print(all_fnames_stokes)
    
    ### Get the HMI Milne-Eddington magentic field inversion series ###
    a_series = attrs.jsoc.Series('hmi.ME_720s_fd10')
    
    if download:
        results_magvec = Fido.search(a_time, a_series, a_notify)
        down_files = Fido.fetch(results_magvec, path=user_dir, max_conn=1)
        # Sort the input names
        all_fnames_magvec = natsort.natsorted(down_files)
    else:
        all_fnames_magvec = parse_folder(dir_path=data_dir, inst='hmi', series='ME_720s_fd10', ext='fits', show=show_files)
        
        if len(all_fnames_magvec) > 1:
            tstamps = [i.split('.')[2] for i in all_fnames_magvec]
            tstamps = [sunpy.time.parse_time('_'.join(i.split('_')[0:2])) for i in tstamps]
            tstamps_diff = [np.abs(i.gps - user_date.gps) for i in tstamps]
        else:
            print('No files found close to the date requested')
            return 
            
        # Search for the closest timestamp
        tstamps_diff = np.asarray(tstamps_diff)
        tstamps_ix, = np.where(tstamps_diff == tstamps_diff.min())

        all_fnames_magvec = np.asarray(all_fnames_magvec)[tstamps_ix]
        
        print(f'No download requested. Nearest inversion {len(all_fnames_magvec)} files found: ')
        print(all_fnames_magvec)
        
    return all_fnames_stokes, all_fnames_magvec
   

# Download HMI timeseries

In [262]:
user_date = astropy.time.Time(datetime.datetime(2016, 7, 28, 23, 57, 0), scale='tai')

all_fnames_stokes, all_fnames_magvec = get_HMI_data(user_date, user_notify='gdima@hawaii.edu', download=True)

Time window used for the search:  <sunpy.net.attrs.Time(2016-07-28 23:56:59.000, 2016-07-28 23:57:01.000)>
Export request pending. [id=JSOC_20210602_265, status=2]
Waiting for 0 seconds...
24 URLs found for download. Full request totalling 246MB


Files Downloaded:   0%|          | 0/24 [00:00<?, ?file/s]

Export request pending. [id=JSOC_20210602_353, status=2]
Waiting for 0 seconds...
25 URLs found for download. Full request totalling 307MB


Files Downloaded:   0%|          | 0/25 [00:00<?, ?file/s]

# Create StokesCube from HMI images

In [250]:
# Obtain a list of available HMI images 
# This needs to be improved to accept user imput for the date
# and type of series wanted. 
# Stokes series is labeled with: S_720s

#all_fnames= parse_folder(dir_path=data_dir, inst='hmi', series='S_720s', ext='fits', show=True)
all_fnames = all_fnames_stokes
all_fnames

array(['/home/gabriel/Desktop/Science/StokesPY/Data/SDO/hmi.s_720s.20160728_234800_TAI.3.I0.fits',
       '/home/gabriel/Desktop/Science/StokesPY/Data/SDO/hmi.s_720s.20160728_234800_TAI.3.I1.fits',
       '/home/gabriel/Desktop/Science/StokesPY/Data/SDO/hmi.s_720s.20160728_234800_TAI.3.I2.fits',
       '/home/gabriel/Desktop/Science/StokesPY/Data/SDO/hmi.s_720s.20160728_234800_TAI.3.I3.fits',
       '/home/gabriel/Desktop/Science/StokesPY/Data/SDO/hmi.s_720s.20160728_234800_TAI.3.I4.fits',
       '/home/gabriel/Desktop/Science/StokesPY/Data/SDO/hmi.s_720s.20160728_234800_TAI.3.I5.fits',
       '/home/gabriel/Desktop/Science/StokesPY/Data/SDO/hmi.s_720s.20160728_234800_TAI.3.Q0.fits',
       '/home/gabriel/Desktop/Science/StokesPY/Data/SDO/hmi.s_720s.20160728_234800_TAI.3.Q1.fits',
       '/home/gabriel/Desktop/Science/StokesPY/Data/SDO/hmi.s_720s.20160728_234800_TAI.3.Q2.fits',
       '/home/gabriel/Desktop/Science/StokesPY/Data/SDO/hmi.s_720s.20160728_234800_TAI.3.Q3.fits',
       '/h

## Create data array

In [251]:
level1_data = []
wcs_obj = []
header_info = []

for i, fname in enumerate(all_fnames):
    with astropy.io.fits.open(fname) as hdulist:
        level1_data.append(hdulist[1].data)
        wcs_obj.append(astropy.wcs.WCS(hdulist[1].header))
        header_info.append(hdulist[1].header)
    
level1_data = np.asarray(level1_data)
level1_data = level1_data.reshape(4,6,level1_data.shape[1], level1_data.shape[2])

print(f'Created data cube with dimensions: {level1_data.shape}')

Created data cube with dimensions: (4, 6, 4096, 4096)


## Create WCS

In [252]:
# Expand the coordinate axis to include wavelength and stokes dimensions.

l0 = 6173.345 * 1.e-10  # m Central wavelength for FeI line
dl = 0.0688   * 1.e-10  # m 

# Generate WCS for data cube using same WCS celestial information from AIA map.
wcs_header = wcs_obj[0].to_header()

wcs_header["WCSAXES"] = 4

# Add wavelength axis.
wcs_header["CRPIX3"] = 3.5
wcs_header["CDELT3"] = dl
wcs_header["CUNIT3"] = 'm'
wcs_header["CTYPE3"] = "WAVE"
wcs_header["CRVAL3"] = l0

# Add Stokes axis.
wcs_header["CRPIX4"] = 0
wcs_header["CDELT4"] = 1
wcs_header["CUNIT4"] = ''
wcs_header["CTYPE4"] = "STOKES"
wcs_header["CRVAL4"] = 0

level1_wcs = astropy.wcs.WCS(wcs_header)

In [253]:
level1_wcs

WCS Keywords

Number of WCS axes: 4
CTYPE : 'HPLN-TAN'  'HPLT-TAN'  'WAVE'  'STOKES'  
CRVAL : 0.0  0.0  6.173345e-07  0.0  
CRPIX : 2039.1322021484  2051.4501953125  3.5  0.0  
PC1_1 PC1_2 PC1_3 PC1_4  : -0.99999997184728  0.00023728765089914  0.0  0.0  
PC2_1 PC2_2 PC2_3 PC2_4  : -0.00023728765089914  -0.99999997184728  0.0  0.0  
PC3_1 PC3_2 PC3_3 PC3_4  : 0.0  0.0  1.0  0.0  
PC4_1 PC4_2 PC4_3 PC4_4  : 0.0  0.0  0.0  1.0  
CDELT : 0.00014009374711249  0.00014009374711249  6.88e-12  1.0  
NAXIS : 0  0

In [145]:
level1_wcs.world_axis_units

['deg', 'deg', 'm', '']

## Create the StokesCube object

In [254]:
level1_cube = stokespy.StokesCube(level1_data, level1_wcs)

In [255]:
# Spectral axis checks out.
print('Input wavelength ') 
print((l0 + dl * (np.arange(0,6) - 3.5 + 1)))
print('StokesCube wavelength')
print(level1_cube.spectral_axis)

Input wavelength 
[6.1731730e-07 6.1732418e-07 6.1733106e-07 6.1733794e-07 6.1734482e-07
 6.1735170e-07]
StokesCube wavelength
[6.1731730e-07 6.1732418e-07 6.1733106e-07 6.1733794e-07 6.1734482e-07
 6.1735170e-07] m


# Create MagVectorCube from HMI inversions

In [256]:
# Obtain a list of available HMI images 
# This needs to be improved to accept user imput for the date
# and type of series wanted. 
# Stokes series is labeled with: S_720s

#all_fnames = parse_folder(dir_path=data_dir, inst='hmi', series='me_720s_fd10', ext='fits', show=True)

all_fnames = all_fnames_magvec
all_fnames

array(['/home/gabriel/Desktop/Science/StokesPY/Data/SDO/hmi.me_720s_fd10.20160728_234800_TAI.alpha_err.fits',
       '/home/gabriel/Desktop/Science/StokesPY/Data/SDO/hmi.me_720s_fd10.20160728_234800_TAI.alpha_mag.fits',
       '/home/gabriel/Desktop/Science/StokesPY/Data/SDO/hmi.me_720s_fd10.20160728_234800_TAI.azimuth.fits',
       '/home/gabriel/Desktop/Science/StokesPY/Data/SDO/hmi.me_720s_fd10.20160728_234800_TAI.azimuth_alpha_err.fits',
       '/home/gabriel/Desktop/Science/StokesPY/Data/SDO/hmi.me_720s_fd10.20160728_234800_TAI.azimuth_err.fits',
       '/home/gabriel/Desktop/Science/StokesPY/Data/SDO/hmi.me_720s_fd10.20160728_234800_TAI.chisq.fits',
       '/home/gabriel/Desktop/Science/StokesPY/Data/SDO/hmi.me_720s_fd10.20160728_234800_TAI.confid_map.fits',
       '/home/gabriel/Desktop/Science/StokesPY/Data/SDO/hmi.me_720s_fd10.20160728_234800_TAI.conv_flag.fits',
       '/home/gabriel/Desktop/Science/StokesPY/Data/SDO/hmi.me_720s_fd10.20160728_234800_TAI.damping.fits',
       

## Create data array

In [257]:
mag_params = ['field', 'inclination', 'azimuth']

level2_data = []
wcs_obj = []
header_info = []

# Load 2D maps into level2_data in the order determined by entries in mag_params
use_fnames = []
for mag_param in mag_params:
    for i, fname in enumerate(all_fnames):
        data_id = fname.split('.')[-2]
        if data_id == mag_param:
            use_fnames.append(fname)
            with astropy.io.fits.open(fname) as hdulist:
                level2_data.append(hdulist[1].data)
                wcs_obj.append(astropy.wcs.WCS(hdulist[1].header))
                header_info.append(hdulist[1].header)

level2_data = np.asarray(level2_data)
                
print(f'Created data cube with dimensions: {level2_data.shape}')
print('Filenames used: ')
for fname in use_fnames:
    print(fname)

Created data cube with dimensions: (3, 4096, 4096)
Filenames used: 
/home/gabriel/Desktop/Science/StokesPY/Data/SDO/hmi.me_720s_fd10.20160728_234800_TAI.field.fits
/home/gabriel/Desktop/Science/StokesPY/Data/SDO/hmi.me_720s_fd10.20160728_234800_TAI.inclination.fits
/home/gabriel/Desktop/Science/StokesPY/Data/SDO/hmi.me_720s_fd10.20160728_234800_TAI.azimuth.fits


## Create WCS

In [258]:
# Expand the wcs coordinates to include the magnetic field parameters.

# Generate WCS for data cube using same WCS celestial information from AIA map.
wcs_header = wcs_obj[0].to_header()

wcs_header["WCSAXES"] = 3

# Add Magnetic field axis.
wcs_header["CRPIX3"] = 0
wcs_header["CDELT3"] = 1
wcs_header["CUNIT3"] = ''
wcs_header["CTYPE3"] = "Parameter"
wcs_header["CRVAL3"] = 0

level2_wcs = astropy.wcs.WCS(wcs_header)

## Create the MagVectorCube object

In [259]:
level2_cube = stokespy.MagVectorCube(level2_data, level2_wcs)

In [260]:
# I use jupyter lab which uses this widget command for interactive plotting
#%matplotlib notebook
%matplotlib widget  

mag_map = level2_cube.B
mag_map.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<WCSAxesSubplot:>